<a href="https://colab.research.google.com/github/rayanebitout-debug/dataset/blob/main/Mini_Projet_Architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
print('======================================================================================')
print('MINI-PROJET DE LA MATIERE FTP')
print("Groupe d'étudiants en Architecture :\n"
      "- SAOU Youssra\n"
      "- OUADIA Imane\n"
      "- BITOUT Rayane\n"
      "- TATAH Lina")
print('======================================================================================')

# ------------------------------------------------------------
# ÉTAPE 0 : Installation des bibliothèques nécessaires
# ------------------------------------------------------------
print("=" * 60)
print("ÉTAPE 0 : Installation des bibliothèques nécessaires")
print("=" * 60)

!pip install seaborn -q
!pip install patool -q  # Pour gérer les fichiers .rar

# ------------------------------------------------------------
# ÉTAPE 1 : Importation des bibliothèques
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 1 : Importation des bibliothèques")
print("=" * 60)

import os
import zipfile
import patoolib
import pandas as pd
import numpy as np
import sqlite3
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

%matplotlib inline

print("✅ Bibliothèques importées avec succès")

# ------------------------------------------------------------
# ÉTAPE 2 : Configuration de l'environnement
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 2 : Configuration de l'environnement")
print("=" * 60)

BASE_DIR = "/content"
DATA_DIR = os.path.join(BASE_DIR, "dataset")
RESULTS_DIR = os.path.join(BASE_DIR, "resultats")

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

print(f"📁 Répertoire données: {DATA_DIR}")
print(f"📁 Répertoire résultats: {RESULTS_DIR}")

# ------------------------------------------------------------
# ÉTAPE 3 : Téléchargement du dataset
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 3 : Téléchargement du dataset")
print("=" * 60)

RAR_PATH = os.path.join(DATA_DIR, "Gainesville.rar")
DATASET_URL = "https://github.com/rayanebitout-debug/dataset/raw/main/Gainesville.rar"

print(f"🔗 URL du dataset: {DATASET_URL}")

# Supprimer l'ancien fichier s'il existe
if os.path.exists(RAR_PATH):
    os.remove(RAR_PATH)

# Télécharger le dataset
!wget --no-check-certificate -q --show-progress "$DATASET_URL" -O "$RAR_PATH"

# Vérifier le téléchargement
if not os.path.exists(RAR_PATH):
    raise RuntimeError("❌ Échec du téléchargement")

file_size = os.path.getsize(RAR_PATH)
print(f"📊 Taille du fichier: {file_size} octets")
print("✅ Dataset téléchargé avec succès")

# ------------------------------------------------------------
# ÉTAPE 4 : Décompression du fichier
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 4 : Décompression du fichier")
print("=" * 60)

try:
    patoolib.extract_archive(RAR_PATH, outdir=DATA_DIR)
    print("✅ Fichier RAR extrait avec succès")
except Exception as e:
    print(f"⚠️ Erreur avec patool: {e}")

    try:
        with zipfile.ZipFile(RAR_PATH, 'r') as zip_ref:
            zip_ref.extractall(DATA_DIR)
            print("✅ Extraction avec zipfile réussie")
    except:
        raise RuntimeError("❌ Impossible d'extraire le fichier")

# ------------------------------------------------------------
# ÉTAPE 5 : Lecture des données
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 5 : Lecture des données")
print("=" * 60)

# Rechercher les fichiers CSV
csv_files = []
for root, dirs, files in os.walk(DATA_DIR):
    for f in files:
        if f.lower().endswith(".csv"):
            csv_files.append(os.path.join(root, f))

if not csv_files:
    raise FileNotFoundError("❌ Aucun fichier CSV trouvé")

csv_path = csv_files[0]
print(f"📄 Fichier CSV trouvé: {csv_path}")

# Essayer différents encodages
encodings = ['utf-8', 'latin-1', 'ISO-8859-1', 'cp1252']
df = None

for encoding in encodings:
    try:
        df = pd.read_csv(csv_path, encoding=encoding)
        print(f"✅ Lecture réussie avec l'encodage: {encoding}")
        break
    except UnicodeDecodeError:
        continue
    except Exception as e:
        print(f"⚠️ Erreur avec {encoding}: {str(e)[:50]}")

if df is None:
    df = pd.read_csv(csv_path, encoding='latin-1', engine='python', on_bad_lines='skip')
    print("⚠️ Lecture avec paramètres de secours")

print("📊 Aperçu des données chargées:")
print(df.head())
print(f"\n📈 Dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")
df_original = df.copy()

# ------------------------------------------------------------
# ÉTAPE 6 : Nettoyage des données
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 6 : Nettoyage des données")
print("=" * 60)

print("🧹 Remplacement des valeurs manquantes...")
missing_values = ['No Value', 'Unknown', 'N/A', 'NA', '', '-', '--', 'null', 'NULL', 'NaN', 'nan', 'None', 'NONE']
df = df.replace(missing_values, np.nan)

print("🔢 Conversion des colonnes numériques...")
potential_numeric_cols = [
    'electric_eui', 'fuel_eui', 'site_eui', 'source_eui',
    'ghg_emissions_int', 'number_of_people', 'occupant_density',
    'operating_hours', 'energy_star_rating', 'leed_score',
    'total_ghg_emissions', 'electricity_use', 'natural_gas_use',
    'total_energy_use', 'building_area', 'year_built'
]

for col in potential_numeric_cols:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        non_null = df[col].notna().sum()
        if non_null > 0:
            print(f"  ✓ {col}: {non_null} valeurs non-nulles")

print("📉 Suppression des colonnes avec trop de valeurs manquantes...")
initial_cols = df.shape[1]
df_cleaned = df.loc[:, df.isna().mean() < 0.9]
df = df_cleaned
removed_cols = initial_cols - df.shape[1]
print(f"  ✓ {removed_cols} colonnes supprimées")

print("🗑️ Suppression des lignes entièrement vides...")
initial_rows = df.shape[0]
df = df.dropna(how='all')
removed_rows = initial_rows - df.shape[0]
print(f"  ✓ {removed_rows} lignes supprimées")

clean_csv = os.path.join(RESULTS_DIR, "Gainesville_clean.csv")
df.to_csv(clean_csv, index=False, encoding='utf-8')
print(f"💾 Données nettoyées sauvegardées: {clean_csv}")
print(f"📊 Nouvelles dimensions: {df.shape[0]} lignes × {df.shape[1]} colonnes")

# ------------------------------------------------------------
# ÉTAPE 7 : Stockage des résultats en base de données
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 7 : Stockage des résultats en base de données")
print("=" * 60)

db_path = os.path.join(RESULTS_DIR, "gainesville.db")
conn = sqlite3.connect(db_path)
df.to_sql("gainesville_energy", conn, if_exists="replace", index=False)
conn.close()

print(f"🗄️ Base de données créée: {db_path}")
print(f"📊 Table 'gainesville_energy' avec {df.shape[0]} enregistrements")

# ------------------------------------------------------------
# ÉTAPE 8 : Visualisation des données
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 8 : Visualisation des données")
print("=" * 60)

# 1. Distribution de l'EUI électrique
if 'electric_eui' in df.columns and df['electric_eui'].notna().sum() > 0:
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.hist(df['electric_eui'].dropna(), bins=40, edgecolor='black', alpha=0.7, color='skyblue')
    plt.title("Distribution Electric EUI", fontsize=14)
    plt.xlabel("kWh/m²")
    plt.ylabel("Nombre de bâtiments")
    plt.grid(alpha=0.3)

    plt.subplot(1, 2, 2)
    df['electric_eui'].dropna().plot(kind='box')
    plt.title("Boxplot Electric EUI", fontsize=14)
    plt.ylabel("kWh/m²")
    plt.grid(alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, "electric_eui_distribution.png"), dpi=300)
    plt.show()
    print("✅ Graphique Electric EUI généré")

# 2. Matrice de corrélation
numeric_df = df.select_dtypes(include=[np.number])
if numeric_df.shape[1] > 2:
    plt.figure(figsize=(10, 8))
    correlation = numeric_df.corr()
    sns.heatmap(correlation, annot=True, fmt='.2f', cmap='coolwarm', center=0)
    plt.title("Matrice de corrélation", fontsize=14)
    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, "correlation_matrix.png"), dpi=300)
    plt.show()
    print("✅ Matrice de corrélation générée")

# 3. Top 10 des bâtiments par consommation
if 'electric_eui' in df.columns and 'building_name' in df.columns:
    top_buildings = df.dropna(subset=['electric_eui']).nlargest(10, 'electric_eui')
    if len(top_buildings) > 0:
        plt.figure(figsize=(12, 6))
        plt.barh(range(len(top_buildings)), top_buildings['electric_eui'])
        plt.yticks(range(len(top_buildings)), top_buildings['building_name'])
        plt.xlabel('Electric EUI (kWh/m²)')
        plt.title('Top 10 bâtiments par consommation électrique')
        plt.gca().invert_yaxis()
        plt.tight_layout()
        plt.savefig(os.path.join(RESULTS_DIR, "top_buildings_eui.png"), dpi=300)
        plt.show()
        print("✅ Graphique top 10 bâtiments généré")

# ------------------------------------------------------------
# ÉTAPE 9 : Génération du rapport détaillé
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 9 : Génération du rapport détaillé")
print("=" * 60)

print("📊 RAPPORT D'ANALYSE - DONNÉES ÉNERGÉTIQUES GAINESVILLE")
print("=" * 50)

print(f"📅 Date d'analyse: {datetime.now().strftime('%d/%m/%Y %H:%M')}")
print(f"📁 Source: {os.path.basename(csv_path)}")
print(f"📈 Données initiales: {df_original.shape[0]} lignes, {df_original.shape[1]} colonnes")
print(f"🧹 Données après nettoyage: {df.shape[0]} lignes, {df.shape[1]} colonnes")
print(f"📉 Données supprimées: {df_original.shape[0] - df.shape[0]} lignes")

print("\n📋 STATISTIQUES PRINCIPALES:")
print("-" * 30)

numeric_cols = df.select_dtypes(include=[np.number]).columns
if len(numeric_cols) > 0:
    print(f"Colonnes numériques: {len(numeric_cols)}")
    for col in numeric_cols[:3]:
        print(f"\n{col}:")
        print(f"  • Moyenne: {df[col].mean():.2f}")
        print(f"  • Médiane: {df[col].median():.2f}")
        print(f"  • Écart-type: {df[col].std():.2f}")

    if 'electric_eui' in df.columns:
        print(f"\n⚡ Consommation électrique moyenne: {df['electric_eui'].mean():.1f} kWh/m²")

cat_cols = df.select_dtypes(include=['object']).columns
if len(cat_cols) > 0:
    print(f"\nColonnes catégorielles: {len(cat_cols)}")

# Sauvegarde du rapport dans un fichier
report_path = os.path.join(RESULTS_DIR, "rapport_analyse.txt")
with open(report_path, "w", encoding="utf-8") as f:
    f.write("RAPPORT D'ANALYSE - DONNÉES ÉNERGÉTIQUES GAINESVILLE\n")
    f.write("=" * 50 + "\n\n")
    f.write(f"Date d'analyse: {datetime.now().strftime('%d/%m/%Y %H:%M')}\n")
    f.write(f"Données initiales: {df_original.shape[0]} lignes × {df_original.shape[1]} colonnes\n")
    f.write(f"Données nettoyées: {df.shape[0]} lignes × {df.shape[1]} colonnes\n\n")

    f.write("RÉSUMÉ STATISTIQUE:\n")
    for col in numeric_cols:
        f.write(f"\n{col}:\n")
        f.write(f"  • Non-null: {df[col].notna().sum()}/{len(df)}\n")
        if df[col].notna().sum() > 0:
            f.write(f"  • Moyenne: {df[col].mean():.2f}\n")
            f.write(f"  • Médiane: {df[col].median():.2f}\n")

print(f"\n💾 Rapport sauvegardé: {report_path}")

# ------------------------------------------------------------
# ÉTAPE 10 : Exportation des résultats dans différents formats
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ÉTAPE 10 : Exportation des résultats")
print("=" * 60)

# Export JSON
json_path = os.path.join(RESULTS_DIR, "gainesville_data.json")
df.head(50).to_json(json_path, orient='records', indent=2)
print(f"📄 JSON exporté: {json_path}")

# Export Excel
excel_path = os.path.join(RESULTS_DIR, "gainesville_analysis.xlsx")
with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Données nettoyées', index=False)

    summary_df = pd.DataFrame({
        'Colonne': df.columns,
        'Type': df.dtypes.astype(str),
        'Non-Null': df.notna().sum().values,
        'Null': df.isna().sum().values,
        '% Null': (df.isna().mean() * 100).round(2).values
    })
    summary_df.to_excel(writer, sheet_name='Résumé', index=False)

print(f"📊 Excel exporté: {excel_path}")

# ------------------------------------------------------------
# FIN : Résumé des fichiers produits
# ------------------------------------------------------------
print("\n" + "=" * 60)
print("ANALYSE TERMINÉE AVEC SUCCÈS")
print("=" * 60)

print("\n📂 FICHIERS PRODUITS:")
print("-" * 30)

files_produced = []
for file in os.listdir(RESULTS_DIR):
    file_path = os.path.join(RESULTS_DIR, file)
    size_kb = os.path.getsize(file_path) / 1024
    files_produced.append((file, size_kb))
    print(f"  • {file} ({size_kb:.1f} KB)")

print(f"\n🎯 Nombre total de fichiers: {len(files_produced)}")
print("📁 Tous les fichiers sont disponibles dans le dossier /content/resultats/")

print("\n" + "=" * 60)
print("MINI-PROJET RÉALISÉ PAR NOTRE GROUPE D'ÉTUDIANTS")
print("Analyse Automatique de Données de Recherche")
print("Sciences et Technologies (ST)")
print("=" * 60)

📊 Excel exporté: /content/resultats/gainesville_analysis.xlsx

ANALYSE TERMINÉE AVEC SUCCÈS

📂 FICHIERS PRODUITS:
------------------------------
  • rapport_detaille.txt (3.4 KB)
  • rapport_analyse.txt (1.0 KB)
  • Gainesville_clean.csv (35119.5 KB)
  • correlation_matrix.png (324.5 KB)
  • gainesville_data.json (27.8 KB)
  • electric_eui_distribution.png (142.6 KB)
  • gainesville.db (33944.0 KB)
  • gainesville_analysis.xlsx (14128.3 KB)

🎯 Nombre total de fichiers: 8
📁 Tous les fichiers sont disponibles dans le dossier /content/resultats/

MINI-PROJET RÉALISÉ PAR NOTRE GROUPE D'ÉTUDIANTS
Analyse Automatique de Données de Recherche
Sciences et Technologies (ST)
